In [ ]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

# 교차 검증

- 고정된 학습과 테스트 데이터로 평가를 하다보면 테스트 데이터에만 최적의 성능을 낸다. 
- 데이타 편중 및 과적합(Overfitting) 문제점
- 본고사를 치르기전에 모의 고사를 여러번 보는 것과 같은 효과 
- K 폴드, Stratified 폴드
- cross_val_score( )



## K 폴드 
- 가장 보편적으로 사용되는 교차 검증 기법 
- 먼저 K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행
``` 
 - 폴더 세트 설정 
 - for 루프에서 반복적으로 학습과 테스트 데이터의 인덱스 추출 
 - 반복적으로 학습과 예측을 수행하고 예측 성능을 반환한다
```


#### 수행 과정 

```
(1) 데이타와 라벨로 분리 

(2) 모델 생성 

(3) kfold 객체 생성 
n 은 폴더 갯수 
kfold = KFold(n_splits=n)

(4) KFold객체의 split( ) 호출 + for 반복문 
# 정확도 리스트 
cv_accuracy = []

# 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 모델 학습 및 예측 
    예측기.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정해서 정확도 리스트에 추가 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    cv_accuracy.append(accuracy)

# 평균 정확도는?
np.mean(cv_accuracy)

```


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

###  5개로 kfold를 지정해서 모델 생성 후 테스트 

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
features = iris.data
label = iris.target

In [ ]:
# model_df = DecisionTreeClassifier(random_state=11)
model_kn = KNeighborsClassifier()

kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])

n_iter = 0

for train_index, test_index  in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    
    model_kn.fit(X_train , y_train)    
    pred = model_kn.predict(X_test)
    n_iter += 1
    
    
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)
    

print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 

# Stratified K 폴드

- 데이타의 분포도가 유사해야한다는 kFold의 특징을 보완 
- 불균형한 분포도를 가진 레이블 데이타 집합을 위한 k폴드 방식 
- 불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 레이블의 값의 분포가 한쪽으로 치우친다.
- 예) 대출사기 데이타. 대출사기 레이블이 전체의 0.0001% 에 밖에 해당하지 못한다.  
- 분류 모델에는 적합하나 회기에서는 연속된 숫자값이기 때문에 적합하지 못하다.


#### 수행 과정 

from sklearn.model_selection import StratifiedKFold

```
(1) 데이타와 라벨로 분리 

(2) 모델 생성 

(3) StratifiedKFold 객체 생성 

n 은 폴더 갯수 
skf = StratifiedKFold(n_splits=n)

(4) StratifiedKFold의 split( ) 호출 + for 반복문 
# 정확도 리스트 
cv_accuracy = []

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 모델 학습 및 예측 
    예측기.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정해서 정확도 리스트에 추가 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    cv_accuracy.append(accuracy)

# 평균 정확도는?
np.mean(cv_accuracy)

```

###  kfold의 문제점 파악하기 

In [ ]:
kfold = KFold(n_splits=5)

n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('\n## 교차 검증: {0}'.format(n_iter))
    print('\n학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('\n검증 레이블 데이터 분포:\n', label_test.value_counts())
    print('='*50)
    

### StratifiedKFold 활용 => 레벨의 분포를 고르게 한다 

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('\n## 교차 검증: {0}'.format(n_iter))
    print('\n학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('\n검증 레이블 데이터 분포:\n', label_test.value_counts())
    print('='*50)

##### Stratified 폴더를 이용한 모델 생성과 정확도 분석 

In [ ]:
model_kn =  KNeighborsClassifier()

skfold = StratifiedKFold(n_splits=5)
n_iter=0
cv_accuracy=[]

for train_index, test_index  in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    model_kn.fit(X_train , y_train)    
    pred = model_kn.predict(X_test)

    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)

print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 

# cross_val_score( )

- 사이킷런에서 제공하는 교차 검증을 편리하게 수행할 수 있게 도와주는 API 
```
cross_val_score(모델 , X_data , y , scoring='accuracy',  cv=폴더갯수)
```



In [ ]:
from sklearn.model_selection import cross_val_score 

In [ ]:
iris_data = load_iris()
model_kn =  KNeighborsClassifier()

data = iris_data.data
label = iris_data.target

scores = cross_val_score(model_kn , data , label , scoring='accuracy', cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))


# GridSearchCV


```
(1) 딕셔너리 형태로 파라미터를 지정 
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]}

(2) GridSearchCV 메서드 적용 
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

```

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=11)

In [ ]:
model_kn = KNeighborsClassifier()

In [ ]:
model_kn.get_params()

In [ ]:
list(range(3,10))

In [ ]:
model_kn = KNeighborsClassifier()

parameters = {'n_neighbors':list(range(3,10))}


model_kn_grid = GridSearchCV(model_kn, param_grid=parameters, cv=5, return_train_score=True)


model_kn_grid.fit(X_train, y_train)

In [ ]:
print(dir(model_kn_grid))

In [ ]:
print('GridSearchCV 최적 파라미터:', model_kn_grid.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(model_kn_grid.best_score_))

In [ ]:
scores_df = pd.DataFrame(model_kn_grid.cv_results_)
scores_df

In [ ]:
print(f'GridSearchCV 최적 파라미터: {model_kn_grid.best_params_}' )
print(f'GridSearchCV 최고 정확도: {model_kn_grid.best_score_}')

# 퀴즈

- 1) 'titanic_book.csv' 타이타닉 데이타셋을 아래와 같은 전처리 후 cross_val_score() 메서드를 이용하여 교차 검증한 후 
<br> 평균 정확도를 구하여라. 모델의 알고리즘은 임의선정, cv는 5 

cross_val_score(model_kn, X_titanic_df , y_titanic_df , cv=5)


- 2) 1)의 데이타셋을 활용하여 GridSearchCV의 최적 하이퍼 파라미터를 찾은 후 정확도를 구하여라. 

parameters = {'n_neighbors':list(range(3,10))}

GridSearchCV(model_kn , param_grid=parameters , scoring='accuracy' , cv=5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

titanic_df = pd.read_csv('data/titanic_book.csv')


titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)


encoder = LabelEncoder()
encoder.fit(titanic_df['Sex'])
temp = encoder.transform(titanic_df['Sex'])
titanic_df['Sex'] = temp.reshape(-1, 1)

encoder.fit(titanic_df['Embarked'])
temp = encoder.transform(titanic_df['Embarked'])
titanic_df['Embarked'] = temp.reshape(-1, 1)

titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
encoder.fit(titanic_df['Cabin'])
temp = encoder.transform(titanic_df['Cabin'])
titanic_df['Cabin'] = temp.reshape(-1, 1)


titanic_df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

In [ ]:
titanic_df.head()